# Democratic Republic of Congo (DRC) Humanitarian Dashboard

Will Geary 

wcg2111@columbia.edu | [willgeary.github.io](willgeary.github.io)

This python notebook contains all of the code used to parse, format and explore the data behind my DRC humanitarian dashboard.

## Conflict Map

*Source: Armed Conflict Location and Event Dataset (ACLED)*

*URL: http://www.acleddata.com/data/realtime-data*

*File: Realtime 2017 All Africa File (updated 18th February 2017)*

Download zip file from url, unzip it, read it into a data frame, and preview the data frame.

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from urllib2 import urlopen

# Plotly libraries for charts
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf
cf.set_config_file(offline=False, world_readable=True) #, theme='ggplot')

# For saving plots offline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [85]:
URL = 'http://www.acleddata.com/wp-content/uploads/2017/02/ACLED-All-Africa-File_20170101-to-20170218_csv.zip'

# open and save the zip file onto computer
url = urlopen(URL)
output = open('data/ACLED_All_Africa_2017.zip', 'wb')    # note the flag:  "wb"        
output.write(url.read())
output.close()

# read the zip file as a pandas dataframe
df = pd.read_csv('data/ACLED_All_Africa_2017.zip')

df.head(3)

,GWNO,EVENT_ID_CNTY,EVENT_ID_NO_CNTY,EVENT_DATE,YEAR,TIME_PRECISION,EVENT_TYPE,ACTOR1,ALLY_ACTOR_1,INTER1,...,ADMIN1,ADMIN2,ADMIN3,LOCATION,LATITUDE,LONGITUDE,GEO_PRECISION,SOURCE,NOTES,FATALITIES
0,615,1416RTA,NaN,22/04/2001,2001,1,Violence against civilians,Police Forces of Algeria (1999-),NaN,1,...,Tizi Ouzou,Beni-Douala,NaN,Beni Douala,36.61954,4.08282,1,Associated Press Online,A Berber student died in police custody at a p...,1
1,615,1417RTA,NaN,23/04/2001,2001,1,Riots/Protests,Rioters (Algeria),Berber Ethnic Group (Algeria),5,...,Tizi Ouzou,Tizi Ouzou,NaN,Tizi Ouzou,36.71183,4.04591,3,Associated Press Online,Five people were killed in 3 days of rioting a...,1
2,615,1418RTA,NaN,24/04/2001,2001,1,Riots/Protests,Rioters (Algeria),Berber Ethnic Group (Algeria),5,...,Tizi Ouzou,Tizi Ouzou,NaN,Tizi Ouzou,36.71183,4.04591,3,Associated Press Online,Five people were killed in 3 days of rioting a...,2


This dataset contains data from many African countries. 

We only are interested in the DRC here, so filter the dataset to only include rows where `COUNTRY` = 'Democratic Republic of Congo'

In [86]:
df = df[df['COUNTRY'] == 'Democratic Republic of Congo']
df.head(3)

,GWNO,EVENT_ID_CNTY,EVENT_ID_NO_CNTY,EVENT_DATE,YEAR,TIME_PRECISION,EVENT_TYPE,ACTOR1,ALLY_ACTOR_1,INTER1,...,ADMIN1,ADMIN2,ADMIN3,LOCATION,LATITUDE,LONGITUDE,GEO_PRECISION,SOURCE,NOTES,FATALITIES
405,490,219RTA,NaN,21/07/2016,2016,1,Battle-Non-state actor overtakes territory,Militia (Kamwina Nsapu),NaN,3,...,Kasai-Occidental,Lulua,Dibaya,Tshimbulu,-6.48333,22.85000,1,La Prosp�rit�,Kamwina Nsapu militiamen attacked the offices ...,0
406,490,220RTA,NaN,23/07/2016,2016,1,Strategic development,Militia (Kamwina Nsapu),NaN,3,...,Kasai-Occidental,Kananga,Kananga,Kananga,-5.89583,22.41778,2,La Prosp�rit�,Kamwina Nsapu militiamen ambushed a vehicle be...,0
407,490,221RTA,NaN,03/08/2016,2016,1,Battle-No change of territory,Militia (Kamwina Nsapu),NaN,3,...,Kasai-Occidental,Lulua,Dibaya,Dibataye,-6.08333,22.90000,2,La Prosp�rit�,Kamwina Nsapu militiamen ambushed a train trav...,0


Here is a table that defines all of the columns. Source: http://www.acleddata.com/wp-content/uploads/2017/01/ACLED_Codebook_2017.pdf

In [87]:
from IPython.display import Image
Image(url= "http://i.imgur.com/keuu4vp.png")

Here is table defining the meanings of the `INTER1` and `INTER2` actor types:

In [88]:
Image(url = "http://i.imgur.com/0wwGGbQ.png")

Here is a table defining the meanings of the `INTERACTION` codes:

In [89]:
Image(url = "http://i.imgur.com/bX4kSUN.png")

Let's select only the columns related to: event date, type, actors involved, location, source and fatalities:

In [90]:
columns = [
 'EVENT_DATE',
 'EVENT_TYPE',
 'ACTOR1',
 'ALLY_ACTOR_1',
 'INTER1',
 'ACTOR2',
 'ALLY_ACTOR_2',
 'INTER2',
 'INTERACTION',
 'COUNTRY',
 'LOCATION',
 'LATITUDE',
 'LONGITUDE',
 'SOURCE',
 'NOTES',
 'FATALITIES']

df = df[columns]
df.head(3)

,EVENT_DATE,EVENT_TYPE,ACTOR1,ALLY_ACTOR_1,INTER1,ACTOR2,ALLY_ACTOR_2,INTER2,INTERACTION,COUNTRY,LOCATION,LATITUDE,LONGITUDE,SOURCE,NOTES,FATALITIES
405,21/07/2016,Battle-Non-state actor overtakes territory,Militia (Kamwina Nsapu),NaN,3,Police Forces of Democratic Republic of Congo ...,NaN,1,13,Democratic Republic of Congo,Tshimbulu,-6.48333,22.85000,La Prosp�rit�,Kamwina Nsapu militiamen attacked the offices ...,0
406,23/07/2016,Strategic development,Militia (Kamwina Nsapu),NaN,3,Civilians (Democratic Republic of Congo),Christian Group (Democratic Republic of Congo),7,37,Democratic Republic of Congo,Kananga,-5.89583,22.41778,La Prosp�rit�,Kamwina Nsapu militiamen ambushed a vehicle be...,0
407,03/08/2016,Battle-No change of territory,Militia (Kamwina Nsapu),NaN,3,Police Forces of Democratic Republic of Congo ...,NaN,1,13,Democratic Republic of Congo,Dibataye,-6.08333,22.90000,La Prosp�rit�,Kamwina Nsapu militiamen ambushed a train trav...,0


Let's replace the `INTER1`, `INTER2`, and `INTERACTION` numeric codes with their meanings as defined in the tables above.

Here are all of the possible actor types:

In [91]:
actor_types = {
0: 'NA', # For single actor events, the empty second actor category is coded as “0”.
1: 'Government or mutinous force',
2: 'Rebel force',
3: 'Political militia',
4: 'Ethnic militia',
5: 'Rioters',
6: 'Protesters',
7: 'Civilians',
8: 'External force'}

Here are all of the possible interaction types:

In [92]:
interaction_types = {
10: 'SOLE MILITARY ACTION',
11: 'MILITARY VERSUS MILITARY',
12: 'MILITARY VERSUS REBELS',
13: 'MILITARY VERSUS POLITICAL MILITIA',
14: 'MILITARY VERSUS COMMUNAL MILITIA',
15: 'MILITARY VERSUS RIOTERS',
16: 'MILITARY VERSUS PROTESTERS',
17: 'MILITARY VERSUS CIVILIANS',
18: 'MILITARY VERSUS OTHER',
20: 'SOLE REBEL ACTION (e.g. base establishment)',
22: 'REBELS VERSUS REBELS',
23: 'REBELS VERSUS POLITICAL MILIITA',
24: 'REBELS VERSUS COMMUNAL MILITIA',
25: 'REBELS VERSUS RIOTERS',
26: 'REBELS VERSUS PROTESTERS',
27: 'REBELS VERSUS CIVILIANS',
28: 'REBELS VERSUS OTHERS',
30: 'SOLE POLITICAL MILITIA ACTION',
33: 'POLITICAL MILITIA VERSUS POLITICAL MILITIA',
34: 'POLITICAL MILITIA VERSUS COMMUNAL MILITIA',
35: 'POLITICAL MILITIA VERSUS RIOTERS',
36: 'POLITICAL MILITIA VERSUS PROTESTERS',
37: 'POLITICAL MILITIA VERSUS CIVILIANS',
38: 'POLITICAL MILITIA VERSUS OTHERS',
40: 'SOLE COMMUNAL MILITIA ACTION',
44: 'COMMUNAL MILITIA VERSUS COMMUNAL MILITIA',
45: 'COMMUNAL MILITIA VERSUS RIOTERS',
46: 'COMMUNAL MILITIA VERSUS PROTESTERS',
47: 'COMMUNAL MILITIA VERSUS CIVILIANS',
48: 'COMMUNAL MILITIA VERSUS OTHER',
50: 'SOLE RIOTER ACTION',
55: 'RIOTERS VERSUS RIOTERS',
56: 'RIOTERS VERSUS PROTESTERS',
57: 'RIOTERS VERSUS CIVILIANS',
58: 'RIOTERS VERSUS OTHERS',
60: 'SOLE PROTESTER ACTION',
66: 'PROTESTERS VERSUS PROTESTERS',
67: 'PROTESTERS VERSUS CIVILIANS',
68: 'PROTESTERS VERSUS OTHER',
78: 'OTHER ACTOR VERSUS CIVILIANS',
80: 'SOLE OTHER ACTION'
}

Map all of the `INTER1` numeric codes with their meanings.

In [93]:
actor_type_1_meanings = []
for i in df['INTER1']:
    actor_type_1_meanings.append(actor_types[i])

Map all of the `INTER2` numeric codes with their meanings.

In [94]:
actor_type_2_meanings = []
for i in df['INTER2']:
    actor_type_2_meanings.append(actor_types[i])

Map all of the `INTERACTION` numeric codes with their meanings.

In [95]:
interaction_meanings = []
for i in df['INTERACTION']:
    interaction_meanings.append(interaction_types[i].capitalize())

Update the dataframe to include the meanings instead of the numeric codes.

In [96]:
df['INTER1'] = actor_type_1_meanings
df['INTER2'] = actor_type_2_meanings
df['INTERACTION'] = interaction_meanings

df.head(3)

,EVENT_DATE,EVENT_TYPE,ACTOR1,ALLY_ACTOR_1,INTER1,ACTOR2,ALLY_ACTOR_2,INTER2,INTERACTION,COUNTRY,LOCATION,LATITUDE,LONGITUDE,SOURCE,NOTES,FATALITIES
405,21/07/2016,Battle-Non-state actor overtakes territory,Militia (Kamwina Nsapu),NaN,Political militia,Police Forces of Democratic Republic of Congo ...,NaN,Government or mutinous force,Military versus political militia,Democratic Republic of Congo,Tshimbulu,-6.48333,22.85000,La Prosp�rit�,Kamwina Nsapu militiamen attacked the offices ...,0
406,23/07/2016,Strategic development,Militia (Kamwina Nsapu),NaN,Political militia,Civilians (Democratic Republic of Congo),Christian Group (Democratic Republic of Congo),Civilians,Political militia versus civilians,Democratic Republic of Congo,Kananga,-5.89583,22.41778,La Prosp�rit�,Kamwina Nsapu militiamen ambushed a vehicle be...,0
407,03/08/2016,Battle-No change of territory,Militia (Kamwina Nsapu),NaN,Political militia,Police Forces of Democratic Republic of Congo ...,NaN,Government or mutinous force,Military versus political militia,Democratic Republic of Congo,Dibataye,-6.08333,22.90000,La Prosp�rit�,Kamwina Nsapu militiamen ambushed a train trav...,0


Let's change some of the column names so that they look nicer and are more understandable.

In [97]:
column_names = {
    'EVENT_DATE': 'Date',
    'EVENT_TYPE': 'Event Type',
    'ACTOR1': 'Actor 1',
    'ALLY_ACTOR_1': 'Actor 1 Ally',
    'INTER1': 'Actor 1 Type',
    'ACTOR2': 'Actor 2',
    'ALLY_ACTOR_2': 'Actor 2 Ally',
    'INTER2': 'Actor 2 Type',
    'INTERACTION': 'Interaction Type',
    'COUNTRY': 'Country',
    'LOCATION': 'Location',
    'LATITUDE': 'Latitude',
    'LONGITUDE': 'Longitude',
    'SOURCE': 'Source',
    'NOTES': 'Notes',
    'FATALITIES': 'Fatalities'}

df = df.rename(columns = column_names)
df.head(3)

,Date,Event Type,Actor 1,Actor 1 Ally,Actor 1 Type,Actor 2,Actor 2 Ally,Actor 2 Type,Interaction Type,Country,Location,Latitude,Longitude,Source,Notes,Fatalities
405,21/07/2016,Battle-Non-state actor overtakes territory,Militia (Kamwina Nsapu),NaN,Political militia,Police Forces of Democratic Republic of Congo ...,NaN,Government or mutinous force,Military versus political militia,Democratic Republic of Congo,Tshimbulu,-6.48333,22.85000,La Prosp�rit�,Kamwina Nsapu militiamen attacked the offices ...,0
406,23/07/2016,Strategic development,Militia (Kamwina Nsapu),NaN,Political militia,Civilians (Democratic Republic of Congo),Christian Group (Democratic Republic of Congo),Civilians,Political militia versus civilians,Democratic Republic of Congo,Kananga,-5.89583,22.41778,La Prosp�rit�,Kamwina Nsapu militiamen ambushed a vehicle be...,0
407,03/08/2016,Battle-No change of territory,Militia (Kamwina Nsapu),NaN,Political militia,Police Forces of Democratic Republic of Congo ...,NaN,Government or mutinous force,Military versus political militia,Democratic Republic of Congo,Dibataye,-6.08333,22.90000,La Prosp�rit�,Kamwina Nsapu militiamen ambushed a train trav...,0


Inspect the data types for each column.

In [99]:
df.dtypes

Date                 object
Event Type           object
Actor 1              object
Actor 1 Ally         object
Actor 1 Type         object
Actor 2              object
Actor 2 Ally         object
Actor 2 Type         object
Interaction Type     object
Country              object
Location             object
Latitude            float64
Longitude           float64
Source               object
Notes                object
Fatalities            int64
dtype: object

Convert the date column to the 'datetime' type.

In [102]:
df['Date'] = pd.to_datetime(df['Date'])

In [103]:
df.dtypes

Date                datetime64[ns]
Event Type                  object
Actor 1                     object
Actor 1 Ally                object
Actor 1 Type                object
Actor 2                     object
Actor 2 Ally                object
Actor 2 Type                object
Interaction Type            object
Country                     object
Location                    object
Latitude                   float64
Longitude                  float64
Source                      object
Notes                       object
Fatalities                   int64
dtype: object

In [105]:
df.head(3)

,Date,Event Type,Actor 1,Actor 1 Ally,Actor 1 Type,Actor 2,Actor 2 Ally,Actor 2 Type,Interaction Type,Country,Location,Latitude,Longitude,Source,Notes,Fatalities
405,2016-07-21,Battle-Non-state actor overtakes territory,Militia (Kamwina Nsapu),NaN,Political militia,Police Forces of Democratic Republic of Congo ...,NaN,Government or mutinous force,Military versus political militia,Democratic Republic of Congo,Tshimbulu,-6.48333,22.85000,La Prosp�rit�,Kamwina Nsapu militiamen attacked the offices ...,0
406,2016-07-23,Strategic development,Militia (Kamwina Nsapu),NaN,Political militia,Civilians (Democratic Republic of Congo),Christian Group (Democratic Republic of Congo),Civilians,Political militia versus civilians,Democratic Republic of Congo,Kananga,-5.89583,22.41778,La Prosp�rit�,Kamwina Nsapu militiamen ambushed a vehicle be...,0
407,2016-03-08,Battle-No change of territory,Militia (Kamwina Nsapu),NaN,Political militia,Police Forces of Democratic Republic of Congo ...,NaN,Government or mutinous force,Military versus political militia,Democratic Republic of Congo,Dibataye,-6.08333,22.90000,La Prosp�rit�,Kamwina Nsapu militiamen ambushed a train trav...,0


Save this data frame to a csv.

In [113]:
df.to_csv('data/ACLED-All-Africa-Clean.csv')

Below is a preview of the map of this data built with Carto. Full map [here](https://willgeary.carto.com/builder/a68a3bfc-f8a1-11e6-87d4-0ecd1babdde5/embed). 

In [124]:
from IPython.display import IFrame
IFrame('https://willgeary.carto.com/builder/a68a3bfc-f8a1-11e6-87d4-0ecd1babdde5/embed', width=1000, height=700)

## Population Density

Health Zone Names (*Zones de Santé Noms et Codes des Zones de santé publique*). Date: Feb 22, 2012

Source: https://data.humdata.org/dataset/dr-congo-health-0

This dataset includes geospatial data including geographic zones and populations in the Congo. I want to make a map of **population density** in the Congo, so I imported the data into QGIS and simply divided `Pop` by `Area` to create a new column called `PopDensity`.

Here is what the dataset looks like after I created the `PopDensity` column, which is on the far right-hand side:

In [131]:
pd.read_csv('data/Congo_Health_Zones_Shapefile/health_zones_attribute_table.csv').head()

,ID,PAYS,PROVINCE,CPROV_SNIS,DIST_ADM,CDA_SNIS,TERRITOIRE,CTER_SNIS,DIST_SA,NOM_ZS,CZS_SNIS,SCE_GEO,Cde_ZS_PUC,Cde_DS_PUC,Nom_DS_PUC,Nom_ZS_PUC,Area,Pop,PopDensity
0,129,RDC,Kasai Oriental,11,Sankuru,1103,KATAKO KOMBE,110301,Sankuru,Wembo Nyama,11030103,OMS,11030103,1103,Sankuru,Wembo Nyama,3897.070056,71278.0,18.29
1,120,RDC,Kasai Oriental,11,Sankuru,1103,KOLE,110302,Sankuru,Kole,11030202,OMS,11030202,1103,Sankuru,Kole,3366.464067,0.0,0.00
2,13,RDC,Kasai Oriental,11,Sankuru,1103,LUSAMBO,110306,Sankuru,Pania Mutombo,11030602,OMS,11030602,1103,Sankuru,Pania Mutombo,4894.307919,0.0,0.00
3,17,RDC,Kasai Oriental,11,Sankuru,1103,LUBEFU,110305,Sankuru,Minga,11030502,OMS,11030501,1103,Sankuru,Minga,6624.578874,0.0,0.00
4,84,RDC,Kasai Oriental,11,Sankuru,1103,LUSAMBO,110306,Sankuru,Lusambo,11030601,OMS,11030601,1103,Sankuru,Lusambo,3786.383161,0.0,0.00


I also want to include the names of the populated towns and cities on the population denstity map. For this I will use Natural Earth's well-known and free [populated places](http://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-populated-places/) dataset.

I loaded this into QGIS and performed a spatial join to only select populated places in the DRC.

Here is the map that I created with Carto:

In [31]:
from IPython.display import IFrame
IFrame('https://willgeary.carto.com/builder/495023ac-f93c-11e6-b67e-0e8c56e2ffdb/embed', width=1000, height=500)

## Displaced Persons

Source: The Humanitarian Data Exchange, *DR Congo - Affected Persons Locations*

URL: https://data.humdata.org/dataset/dr-congo-affected-persons-locations

Description: Internally displaced persons, return of population, refugees and returnees in the DRC. Numbers / locations of displaced internally displaced, returned, refugees and returnees in the DRC.

Open the xlsx file from url, read it into a dataframe.

In [30]:
URL = 'https://data.humdata.org/dataset/fce36ec7-d010-4701-8f93-096b5a1e846e/resource/3d9118d1-a224-44f8-80b0-bc4252a6ca2f/download/rdc_synthese_idps25juin2015.xls'

# open and save the file onto computer
url = urlopen(URL)
output = open('data/rdc_synthese_idps25juin2015.xls', 'wb')    # note the flag:  "wb"        
output.write(url.read())
output.close()

# read the xlsx file (second sheet) as a pandas dataframe
df = pd.read_excel('data/rdc_synthese_idps25juin2015.xls',  sheetname='Evolution globale')

In [31]:
# select only the rows and columns that we want
df = df[['Unnamed: 3','Unnamed: 4', 'Unnamed: 5']].drop(df.index[[0,1,2,3,4]])

In [32]:
# rename the columns
column_names = {'Unnamed: 3': 'Quarter', 'Unnamed: 4': 'IDPs', 'Unnamed: 5': 'Returnees'}
df = df.rename(columns = column_names)
df

,Quarter,IDPs,Returnees
5,2012 - 1,2017898,684448
6,2012 - 2,2240254,726483
7,2012 - 3,2435351,972082
8,2012 - 4,2626207,1233684
9,2013 - 1,2586389,1413604
10,2013 - 2,2607407,1703027
11,2013 - 3,2754515,1785444
12,2013 - 4,2963704,1831882
13,2014 - 1,2634872,1809748
14,2014 - 2,2611558,1559201


In [33]:
# export this data to csv
df.to_csv("data/rdc_synthese_idps25juin2015_clean.csv")

In [34]:
trace1 = go.Scatter(
    x=df.Quarter,
    y=df.IDPs,
    name='IDPs'
)

trace2 = go.Scatter(
    x=df.Quarter,
    y=df.Returnees,
    name='Returnees'
)

data = [trace1, trace2]
layout = go.Layout(
    legend=dict(orientation="h", x=0.5,y=0.1),
    title='Internally Displaced Persons and Returnees',
    titlefont=dict(
            size=22),
    xaxis=dict(
        title='Quarter',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='# of Persons',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

# create figure for offline use
fig = dict( data=data, layout=layout )

# plot inline
iplot(fig, show_link=False)

# save figure offline
myChart = plotly.offline.plot(fig,output_type="div", show_link="False",include_plotlyjs="Flase",link_text="")
with open("charts/idp_returnees.html", "w") as text_file:
    text_file.write(myChart)

## Rural vs. Urban Population

Source: Food and Agriculture Organization of the United Nations

URL: http://www.fao.org/faostat/en/#country/250

Read csv into data frame.

In [16]:
df = pd.read_csv("data/FAOSTAT_data_UrbanRural.csv")
df.head(3)

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,OA,Annual population,250,Democratic Republic of the Congo,551,Rural population,3010,Population - Est. & Proj.,1990,1990,1000 persons,24217.000,XUru,"UNDESA, Population Division – World Population..."
1,OA,Annual population,250,Democratic Republic of the Congo,561,Urban population,3010,Population - Est. & Proj.,1990,1990,1000 persons,10694.055,XUru,"UNDESA, Population Division – World Population..."
2,OA,Annual population,250,Democratic Republic of the Congo,561,Urban population,3010,Population - Est. & Proj.,1991,1991,1000 persons,11262.771,XUru,"UNDESA, Population Division – World Population..."


Filter by population type.

In [17]:
df = df[['Element', 'Year', 'Value']]
rural = df[df['Element'] == 'Rural population']
urban = df[df['Element'] == 'Urban population']

In [37]:
trace1 = go.Bar(
    x=rural.Year,
    y=rural.Value,
    name='Rural'
)
trace2 = go.Bar(
    x=urban.Year,
    y=urban.Value,
    name='Urban'
)

data = [trace2, trace1]
layout = go.Layout(
    legend=dict(orientation="h", x=0.05,y=0.8),
    title='Urban vs. Rural Population',
    titlefont=dict(
            size=22),
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='1000s of Persons',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    barmode='stack'
)

# create figure for offline use
fig = dict( data=data, layout=layout )

# plot inline
iplot(fig, show_link=False)

# save figure offline
myChart = plotly.offline.plot(fig,output_type="div", show_link="False",include_plotlyjs="Flase",link_text="")
with open("charts/urban_rural.html", "w") as text_file:
    text_file.write(myChart)

## Fertility Rate

Source: UNESCO

URL: http://data.uis.unesco.org/

Table: Demographic and socio-economic -> Demographic indicators -> Fertility rate, total (births per women)

Let's look at DRC's ferility rate compared to neighboring African countries. Read the table into a data frame.

In [90]:
df = pd.read_csv('data/DEMO_DS_22022017181129462.csv')
df.head()

,"﻿""DEMO_IND""",Indicator,LOCATION,Country,TIME,Time,Value,Flag Codes,Flags
0,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AGO,Angola,1999,1999,6.943,NaN,NaN
1,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AGO,Angola,2000,2000,6.905,NaN,NaN
2,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AGO,Angola,2001,2001,6.866,NaN,NaN
3,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AGO,Angola,2002,2002,6.829,NaN,NaN
4,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AGO,Angola,2003,2003,6.792,NaN,NaN


In [91]:
df = df[['Country', 'Time', 'Value']]

In [92]:
df.head()

,Country,Time,Value
0,Angola,1999,6.943
1,Angola,2000,6.905
2,Angola,2001,6.866
3,Angola,2002,6.829
4,Angola,2003,6.792


Pivot table.

In [93]:
df = df.pivot(index='Time', columns='Country', values='Value')
df.head()

Country,Angola,Burundi,Central African Republic,Congo,Democratic Republic of the Congo,Rwanda,South Sudan,Uganda,Zambia
Time,,,,,,,,,
1999,6.943,7.112,5.488,5.113,7.079,5.734,6.303,6.903,6.116
2000,6.905,7.057,5.446,5.109,7.053,5.640,6.223,6.865,6.097
2001,6.866,7.000,5.393,5.105,7.018,5.545,6.139,6.822,6.080
2002,6.829,6.939,5.329,5.101,6.975,5.449,6.054,6.772,6.064
2003,6.792,6.873,5.257,5.096,6.924,5.353,5.969,6.716,6.048


In [94]:
trace1 = go.Scatter(
    x=df.index,
    y=df.Angola,
    name='Angola',
    mode = 'lines'
)

trace2 = go.Scatter(
    x=df.index,
    y=df.Burundi,
    name='Burundi',
    mode = 'lines'
)

trace3 = go.Scatter(
    x=df.index,
    y=df['Central African Republic'],
    name='Central African Republic',
    mode = 'lines'
)

trace4 = go.Scatter(
    x=df.index,
    y=df.Congo,
    name='Congo',
    mode = 'lines'
)

trace5 = go.Scatter(
    x=df.index,
    y=df['Democratic Republic of the Congo'],
    name='Democratic Republic of the Congo',
    mode = 'lines',
    line = dict(
        width = 4,
        dash = 'dash')
)

trace6 = go.Scatter(
    x=df.index,
    y=df['Rwanda'],
    name='Rwanda',
    mode = 'lines'
)

trace7 = go.Scatter(
    x=df.index,
    y=df['South Sudan'],
    name='South Sudan',
    mode = 'lines'
)

trace8 = go.Scatter(
    x=df.index,
    y=df['Uganda'],
    name='Uganda',
    mode = 'lines'
)

trace9 = go.Scatter(
    x=df.index,
    y=df['Zambia'],
    name='Zambia',
    mode = 'lines'
)

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
layout = go.Layout(showlegend=False,
    title='Fertility Rate (DRC vs Neighbors)',
    titlefont=dict(
            size=22),
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Births per Woman',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

# create figure for offline use
fig = dict( data=data, layout=layout )

# plot inline
iplot(fig, show_link=False)

# save figure offline
myChart = plotly.offline.plot(fig,output_type="div", show_link="False",include_plotlyjs="Flase",link_text="")
with open("charts/fertility_rate.html", "w") as text_file:
    text_file.write(myChart)

## Life Expectancy at Birth

Source: UNESCO

URL: http://data.uis.unesco.org/

Table: Demographic and socio-economic -> Demographic indicators -> Life expectancy at birth, total (years)

Let's look at DRC's life expectancy compared to neighboring African countries. Read the table into a data frame.

In [95]:
df = pd.read_csv('data/DEMO_DS_22022017184210236.csv')
df.head()

,"﻿""DEMO_IND""",Indicator,LOCATION,Country,TIME,Time,Value,Flag Codes,Flags
0,SP_DYN_LE00_IN,"Life expectancy at birth, total (years)",AGO,Angola,1999,1999,44.43900,NaN,NaN
1,SP_DYN_LE00_IN,"Life expectancy at birth, total (years)",AGO,Angola,2000,2000,45.20478,NaN,NaN
2,SP_DYN_LE00_IN,"Life expectancy at birth, total (years)",AGO,Angola,2001,2001,45.97315,NaN,NaN
3,SP_DYN_LE00_IN,"Life expectancy at birth, total (years)",AGO,Angola,2002,2002,46.71173,NaN,NaN
4,SP_DYN_LE00_IN,"Life expectancy at birth, total (years)",AGO,Angola,2003,2003,47.39317,NaN,NaN


In [96]:
df = df[['Country', 'Time', 'Value']]
df = df.pivot(index='Time', columns='Country', values='Value')
df.head()

Country,Angola,Burundi,Central African Republic,Congo,Democratic Republic of the Congo,Rwanda,South Sudan,Uganda,Zambia
Time,,,,,,,,,
1999,44.43900,51.14924,44.94315,50.62827,49.51741,46.22607,48.80629,45.56498,42.85578
2000,45.20478,51.48549,44.59729,50.58988,50.00288,48.19237,49.21907,46.41998,43.45668
2001,45.97315,51.72483,44.36207,50.77320,50.62934,49.50768,49.61190,47.40415,44.27249
2002,46.71173,51.92827,44.25341,51.17929,51.34680,50.60480,49.99324,48.47559,45.30666
2003,47.39317,52.14532,44.27780,51.79422,52.10678,51.81207,50.37410,49.58780,46.53863


In [97]:
trace1 = go.Scatter(
    x=df.index,
    y=df.Angola,
    name='Angola',
    mode = 'lines'
)

trace2 = go.Scatter(
    x=df.index,
    y=df.Burundi,
    name='Burundi',
    mode = 'lines'
)

trace3 = go.Scatter(
    x=df.index,
    y=df['Central African Republic'],
    name='Central African Republic',
    mode = 'lines'
)

trace4 = go.Scatter(
    x=df.index,
    y=df.Congo,
    name='Congo',
    mode = 'lines'
)

trace5 = go.Scatter(
    x=df.index,
    y=df['Democratic Republic of the Congo'],
    name='Democratic Republic of the Congo',
    mode = 'lines',
    line = dict(
        width = 4,
        dash = 'dash')
)

trace6 = go.Scatter(
    x=df.index,
    y=df['Rwanda'],
    name='Rwanda',
    mode = 'lines'
)

trace7 = go.Scatter(
    x=df.index,
    y=df['South Sudan'],
    name='South Sudan',
    mode = 'lines'
)

trace8 = go.Scatter(
    x=df.index,
    y=df['Uganda'],
    name='Uganda',
    mode = 'lines'
)

trace9 = go.Scatter(
    x=df.index,
    y=df['Zambia'],
    name='Zambia',
    mode = 'lines'
)

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
layout = go.Layout(showlegend=False,
    title='Life Expectancy (DRC vs Neighbors)',
    titlefont=dict(
            size=22),
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Life Expectancy in Years',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

# create figure for offline use
fig = dict( data=data, layout=layout )

# plot inline
iplot(fig, show_link=False)

# save figure offline
myChart = plotly.offline.plot(fig,output_type="div", show_link="False",include_plotlyjs="Flase",link_text="")
with open("charts/life_expectancy.html", "w") as text_file:
    text_file.write(myChart)

## Infant Mortality Rate

Source: UNESCO

URL: http://data.uis.unesco.org/

Table: Demographic and socio-economic -> Demographic indicators -> Infant Mortality Rate (per 1,000 live births)

Let's look at DRC's infant mortality rate compared to neighboring African countries. Read the table into a data frame.

In [98]:
df = pd.read_csv('data/DEMO_DS_22022017181129462.csv')
df.head()

,"﻿""DEMO_IND""",Indicator,LOCATION,Country,TIME,Time,Value,Flag Codes,Flags
0,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AGO,Angola,1999,1999,6.943,NaN,NaN
1,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AGO,Angola,2000,2000,6.905,NaN,NaN
2,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AGO,Angola,2001,2001,6.866,NaN,NaN
3,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AGO,Angola,2002,2002,6.829,NaN,NaN
4,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AGO,Angola,2003,2003,6.792,NaN,NaN


In [99]:
df = df[['Country', 'Time', 'Value']]
df = df.pivot(index='Time', columns='Country', values='Value')
df.head()

Country,Angola,Burundi,Central African Republic,Congo,Democratic Republic of the Congo,Rwanda,South Sudan,Uganda,Zambia
Time,,,,,,,,,
1999,6.943,7.112,5.488,5.113,7.079,5.734,6.303,6.903,6.116
2000,6.905,7.057,5.446,5.109,7.053,5.640,6.223,6.865,6.097
2001,6.866,7.000,5.393,5.105,7.018,5.545,6.139,6.822,6.080
2002,6.829,6.939,5.329,5.101,6.975,5.449,6.054,6.772,6.064
2003,6.792,6.873,5.257,5.096,6.924,5.353,5.969,6.716,6.048


In [100]:
trace1 = go.Scatter(
    x=df.index,
    y=df.Angola,
    name='Angola',
    mode = 'lines'
)

trace2 = go.Scatter(
    x=df.index,
    y=df.Burundi,
    name='Burundi',
    mode = 'lines'
)

trace3 = go.Scatter(
    x=df.index,
    y=df['Central African Republic'],
    name='Central African Republic',
    mode = 'lines'
)

trace4 = go.Scatter(
    x=df.index,
    y=df.Congo,
    name='Congo',
    mode = 'lines'
)

trace5 = go.Scatter(
    x=df.index,
    y=df['Democratic Republic of the Congo'],
    name='Democratic Republic of the Congo',
    mode = 'lines',
    line = dict(
        width = 4,
        dash = 'dash')
)

trace6 = go.Scatter(
    x=df.index,
    y=df['Rwanda'],
    name='Rwanda',
    mode = 'lines'
)

trace7 = go.Scatter(
    x=df.index,
    y=df['South Sudan'],
    name='South Sudan',
    mode = 'lines'
)

trace8 = go.Scatter(
    x=df.index,
    y=df['Uganda'],
    name='Uganda',
    mode = 'lines'
)

trace9 = go.Scatter(
    x=df.index,
    y=df['Zambia'],
    name='Zambia',
    mode = 'lines'
)

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
layout = go.Layout(showlegend=False,
    title='Infant Mortality Rate (DRC vs Neighbors)',
    titlefont=dict(
            size=22),
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Per 1,000 Live Births',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

# create figure for offline use
fig = dict( data=data, layout=layout )

# plot inline
iplot(fig, show_link=False)

# save figure offline
myChart = plotly.offline.plot(fig,output_type="div", show_link="False",include_plotlyjs="Flase",link_text="")
with open("charts/infant_mortality.html", "w") as text_file:
    text_file.write(myChart)

## GDP per Capita

Source: UNESCO

URL: http://data.uis.unesco.org/

Table: Demographic and socio-economic -> Socio-economic indicators -> GDP Per Capita (current US$)

Let's look at DRC's GDP per Capita compared to neighboring African countries. Read the table into a data frame.

In [101]:
df = pd.read_csv("data/DEMO_DS_22022017195753581.csv")
df = df[['Country', 'Time', 'Value']]
df = df.pivot(index='Time', columns='Country', values='Value')
df.head()

Country,Angola,Burundi,Central African Republic,Congo,Democratic Republic of the Congo,Rwanda,South Sudan,Uganda,Zambia
Time,,,,,,,,,
1999,421.37678,121.99773,273.58136,776.36329,100.69324,239.63361,NaN,260.50857,330.21725
2000,606.27229,128.63554,245.43438,1035.58446,397.26487,216.27589,NaN,260.68447,340.16138
2001,574.19416,126.21708,245.56327,877.62639,150.42138,201.06404,NaN,238.05107,376.98106
2002,775.76552,115.27988,256.85061,927.26959,171.23401,196.44470,NaN,243.67462,376.46804
2003,850.07569,106.01697,290.50966,1049.31760,169.90859,212.51205,NaN,241.69480,429.00728


In [102]:
trace1 = go.Scatter(
    x=df.index,
    y=df.Angola,
    name='Angola',
    mode = 'lines'
)

trace2 = go.Scatter(
    x=df.index,
    y=df.Burundi,
    name='Burundi',
    mode = 'lines'
)

trace3 = go.Scatter(
    x=df.index,
    y=df['Central African Republic'],
    name='Central African Republic',
    mode = 'lines'
)

trace4 = go.Scatter(
    x=df.index,
    y=df.Congo,
    name='Congo',
    mode = 'lines'
)

trace5 = go.Scatter(
    x=df.index,
    y=df['Democratic Republic of the Congo'],
    name='Democratic Republic of the Congo',
    mode = 'lines',
    line = dict(
        width = 4,
        dash = 'dash')
)

trace6 = go.Scatter(
    x=df.index,
    y=df['Rwanda'],
    name='Rwanda',
    mode = 'lines'
)

trace7 = go.Scatter(
    x=df.index,
    y=df['South Sudan'],
    name='South Sudan',
    mode = 'lines'
)

trace8 = go.Scatter(
    x=df.index,
    y=df['Uganda'],
    name='Uganda',
    mode = 'lines'
)

trace9 = go.Scatter(
    x=df.index,
    y=df['Zambia'],
    name='Zambia',
    mode = 'lines'
)

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
layout = go.Layout(showlegend=False,
    title='GDP per Capita (DRC vs Neighbors)',
    titlefont=dict(
            size=22),
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Current USD($)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

# create figure for offline use
fig = dict( data=data, layout=layout )

# plot inline
iplot(fig, show_link=False)

# save figure offline
myChart = plotly.offline.plot(fig,output_type="div", show_link="False",include_plotlyjs="Flase",link_text="")
with open("charts/gdp_per_capita.html", "w") as text_file:
    text_file.write(myChart)

## Total GDP

Source: UNESCO

URL: http://data.uis.unesco.org/

Table: Demographic and socio-economic -> Socio-economic indicators -> GDP (current US$)

Let's look at DRC's Total GDP compared to neighboring African countries. 

Read the table into a data frame.

In [103]:
df = pd.read_csv("data/DEMO_DS_22022017200126033.csv")
df = df[['Country', 'Time', 'Value']]
df = df.pivot(index='Time', columns='Country', values='Value')
df.head()

Country,Angola,Burundi,Central African Republic,Congo,Democratic Republic of the Congo,Rwanda,South Sudan,Uganda,Zambia
Time,,,,,,,,,
1999,6.152937e+09,8.080772e+08,9.994775e+08,2.353909e+09,4.711259e+09,1.817655e+09,NaN,5.998563e+09,3.404312e+09
2000,9.129635e+09,8.704861e+08,9.145003e+08,3.219911e+09,1.908805e+10,1.734938e+09,NaN,6.193247e+09,3.600683e+09
2001,8.936064e+09,8.767947e+08,9.318333e+08,2.794260e+09,7.438189e+09,1.674685e+09,NaN,5.840504e+09,4.094481e+09
2002,1.249735e+10,8.253945e+08,9.913879e+08,3.019994e+09,8.728039e+09,1.677447e+09,NaN,6.178564e+09,4.193846e+09
2003,1.418895e+10,7.846544e+08,1.139755e+09,3.495869e+09,8.937567e+09,1.845979e+09,NaN,6.336696e+09,4.901840e+09


In [104]:
trace1 = go.Scatter(
    x=df.index,
    y=df.Angola,
    name='Angola',
    mode = 'lines'
)

trace2 = go.Scatter(
    x=df.index,
    y=df.Burundi,
    name='Burundi',
    mode = 'lines'
)

trace3 = go.Scatter(
    x=df.index,
    y=df['Central African Republic'],
    name='Central African Republic',
    mode = 'lines'
)

trace4 = go.Scatter(
    x=df.index,
    y=df.Congo,
    name='Congo',
    mode = 'lines'
)

trace5 = go.Scatter(
    x=df.index,
    y=df['Democratic Republic of the Congo'],
    name='Democratic Republic of the Congo',
    mode = 'lines',
    line = dict(
        width = 4,
        dash = 'dash')
)

trace6 = go.Scatter(
    x=df.index,
    y=df['Rwanda'],
    name='Rwanda',
    mode = 'lines'
)

trace7 = go.Scatter(
    x=df.index,
    y=df['South Sudan'],
    name='South Sudan',
    mode = 'lines'
)

trace8 = go.Scatter(
    x=df.index,
    y=df['Uganda'],
    name='Uganda',
    mode = 'lines'
)

trace9 = go.Scatter(
    x=df.index,
    y=df['Zambia'],
    name='Zambia',
    mode = 'lines'
)

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
layout = go.Layout(showlegend=False,
    title='Total GDP (DRC vs Neighbors)',
    titlefont=dict(
            size=22),
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Current USD($)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

# create figure for offline use
fig = dict( data=data, layout=layout )

# plot inline
iplot(fig, show_link=False)

# save figure offline
myChart = plotly.offline.plot(fig,output_type="div", show_link="False",include_plotlyjs="Flase",link_text="")
with open("charts/total_gdp.html", "w") as text_file:
    text_file.write(myChart)

## GDP Growth (annual %)

Source: UNESCO

URL: http://data.uis.unesco.org/

Table: Demographic and socio-economic -> Socio-economic indicators -> GDP Growth (annual %)

Let's look at DRC's GDP growth compared to neighboring African countries. 

Read the table into a data frame.

In [105]:
df = pd.read_csv("data/DEMO_DS_22022017200437863.csv")
df = df[['Country', 'Time', 'Value']]
df = df.pivot(index='Time', columns='Country', values='Value')
df.head()

Country,Angola,Burundi,Central African Republic,Congo,Democratic Republic of the Congo,Rwanda,South Sudan,Uganda,Zambia
Time,,,,,,,,,
1999,3.23980,-1.01000,3.60000,-2.58220,-4.27014,7.58110,NaN,8.05395,4.65019
2000,3.01208,-0.85686,-2.48945,7.57598,-6.91093,8.31870,NaN,3.14191,3.89732
2001,4.22097,2.05581,4.46478,3.80260,-2.10017,8.66895,NaN,5.18366,5.31687
2002,-6.94046,4.44652,3.61649,4.58187,2.94777,13.51053,NaN,8.73269,4.50601
2003,5.24749,-1.22373,-5.39747,0.81326,5.57782,1.45134,NaN,6.47326,6.94497


In [106]:
trace1 = go.Scatter(
    x=df.index,
    y=df.Angola,
    name='Angola',
    mode = 'lines'
)

trace2 = go.Scatter(
    x=df.index,
    y=df.Burundi,
    name='Burundi',
    mode = 'lines'
)

trace3 = go.Scatter(
    x=df.index,
    y=df['Central African Republic'],
    name='Central African Republic',
    mode = 'lines'
)

trace4 = go.Scatter(
    x=df.index,
    y=df.Congo,
    name='Congo',
    mode = 'lines'
)

trace5 = go.Scatter(
    x=df.index,
    y=df['Democratic Republic of the Congo'],
    name='Democratic Republic of the Congo',
    mode = 'lines',
    line = dict(
        width = 4,
        dash = 'dash')
)

trace6 = go.Scatter(
    x=df.index,
    y=df['Rwanda'],
    name='Rwanda',
    mode = 'lines'
)

trace7 = go.Scatter(
    x=df.index,
    y=df['South Sudan'],
    name='South Sudan',
    mode = 'lines'
)

trace8 = go.Scatter(
    x=df.index,
    y=df['Uganda'],
    name='Uganda',
    mode = 'lines'
)

trace9 = go.Scatter(
    x=df.index,
    y=df['Zambia'],
    name='Zambia',
    mode = 'lines'
)

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
layout = go.Layout(showlegend=False,
    title='GDP Growth (DRC vs Neighbors)',
    titlefont=dict(
            size=22),
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Annual % Growth',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

# create figure for offline use
fig = dict( data=data, layout=layout )

# plot inline
iplot(fig, show_link=False)

# save figure offline
myChart = plotly.offline.plot(fig,output_type="div", show_link="False",include_plotlyjs="Flase",link_text="")
with open("charts/gdp_growth.html", "w") as text_file:
    text_file.write(myChart)

## Poverty Headcount Ratio at 3.10 dollars a day (% of population)

Source: UNESCO

URL: http://data.uis.unesco.org/

Table: Demographic and socio-economic -> Socio-economic indicators -> Poverty Headcount Ratio at 3.10 dollars a day (% of population)

Let's look at DRC's Poverty Headcount Ratio compared to neighboring African countries. 

Read the table into a data frame.

In [108]:
df = pd.read_csv("data/DEMO_DS_22022017201041975.csv")
df = df[['Country', 'Time', 'Value']]
df = df.pivot(index='Time', columns='Country', values='Value')
df.head()

Country,Angola,Burundi,Central African Republic,Congo,Democratic Republic of the Congo,Rwanda,Uganda,Zambia
Time,,,,,,,,
1999,NaN,NaN,NaN,NaN,NaN,NaN,77.46,NaN
2000,54.25,NaN,NaN,NaN,NaN,89.65,NaN,NaN
2002,NaN,NaN,NaN,NaN,NaN,NaN,82.48,75.08
2003,NaN,NaN,84.17,NaN,NaN,NaN,NaN,NaN
2004,NaN,NaN,NaN,NaN,96.85,NaN,NaN,74.02


In [109]:
df.T

Time,1999,2000,2002,2003,2004,2005,2006,2008,2009,2010,2011,2012
Country,,,,,,,,,,,,
Angola,NaN,54.25,NaN,NaN,NaN,NaN,NaN,54.52,NaN,NaN,NaN,NaN
Burundi,NaN,NaN,NaN,NaN,NaN,NaN,92.17,NaN,NaN,NaN,NaN,NaN
Central African Republic,NaN,NaN,NaN,84.17,NaN,NaN,NaN,82.27,NaN,NaN,NaN,NaN
Congo,NaN,NaN,NaN,NaN,NaN,71.76,NaN,NaN,NaN,NaN,52.91,NaN
Democratic Republic of the Congo,NaN,NaN,NaN,NaN,96.85,NaN,NaN,NaN,NaN,NaN,NaN,90.73
Rwanda,NaN,89.65,NaN,NaN,NaN,84.05,NaN,NaN,NaN,80.66,NaN,NaN
Uganda,77.46,NaN,82.48,NaN,NaN,76.33,NaN,NaN,69.37,NaN,NaN,63.03
Zambia,NaN,NaN,75.08,NaN,74.02,NaN,76.86,NaN,NaN,78.87,NaN,NaN


In [110]:
df.T.to_csv("/Users/Will/Dropbox/Documents/Work/UN/data/poverty-transpose")

## Population Pyramid

I used R to create the below animated population pyramid for the DRC.

Source: International Data Base from the US Census Bureau

URL: https://walkerke.github.io/2014/06/rcharts-pyramids/

In [111]:
from IPython.display import IFrame
IFrame('charts/congo_population.html', width=1000, height=500)